Modelo de McCulloch-Pitts
===

**Juan David Velásquez Henao**  
jdvelasq@unal.edu.co   
Universidad Nacional de Colombia, Sede Medellín  
Facultad de Minas  
Medellín, Colombia

---

Haga click [aquí](https://github.com/jdvelasq/deep-neural-nets/tree/master/) para acceder al repositorio online.

Haga click [aquí](http://nbviewer.jupyter.org/github/jdvelasq/deep-neural-nets/tree/master/) para explorar el repositorio usando `nbviewer`. 

---

# Definición del problema

Se desea construir un sistema de visión que permite reconocer patrones binarios. Para este caso práctico, se desean reconocer los cuatro patrones de la siguiente figura.  

![alt](images/McCullochPitts-01.png)

# Metodología de Solución

El modelo de neurona de McCulloch-Pitts fue propuesto originalmente como un postulado sobre la forma en que el cerebro puede reconocer patrones complejos (parte derecha de la figura anterior). Este modelo plantea que, en general, una célula (neurona) puede representarse matemáticamente como una unidad genérica de cómputo de la siguiente forma:

![alt](images/McCullochPitts-02.png)

donde $x_i$ representa la $i$-ésima entrada binaria, $g()$ es una función de agregación y $f()$ es una función de transformación no lineal, donde:

$$g(x_1, ...,x_n) = v = \sum_{i=1}^n x_i, \qquad f(v) = 
\begin{cases}      
      1, & \text{Si $v \ge \theta$}\\
      0, & \text{Si $v \lt \theta$}\\
\end{cases}$$

Las entradas $x_i$ son señales exitadores, y las señales $y_j$ son inhibidoras. La salida es cero (0) si alguna de las señales inhibidoras es uno (1).  La salida es uno (1) si la suma de señales de entrada es mayor o igual que el umbral ($\theta$), y todas las señales inhibidoras son cero (0).

Una función lógica $y=f(x_1, x_2, ..., x_n)$ se define como $f:\{0,1\}^n \to \{0,1\}$ con:  $y  \in \{0,1\}$, y $x_i \in \{0,1\}$. Ahora, cualquier función lógica $F:\{0,1\}^n \to \{0,1\}$ puede ser calculada por una red McCulloch-Pitts de dos capas.

* Se decodifica para las entradas con $F=1$.


* $\theta$ es igual a la cantidad de 1s en la entrada.


* La neurona de la capa de salida es un OR.

![alt](images/McCullochPitts-03.png)

Todas las funciones lógicas pueden implementarse con una red compuesta de unidades que calculan exclusivamente las funciones AND, OR y NOT. 

---
**Ejercicio.--** Calcule la salida para la siguiente red de neuronas de McCulloch-Pitts.

![alt](images/McCullochPitts-04.png)


**Ejercicio.--** Construya la red de dos capas de McCulloch-Pitts para la función de paridad, y grafique la función de paridad en el hipercubo unitario.

![alt](images/McCullochPitts-05.png)

---

Una neurona de McCulloch-Pitts puede interpretarse como compuerta lógica de umbral (circuito lógico):

![alt](images/McCullochPitts-06.png)

Una neurona de McCulloch-Pitts solo puede representar patrones que sean linealmente separables. Es decir, es posible trazar un plano que separa los puntos con $F=0$ y $F=1$.

![alt](images/McCullochPitts-07.png)

### Solución al problema propuesto

Para el problema propuesto, cada patrón puede ser codificado como un vector de tres posiciones. Cuando el cuadro es negro, el valor de la posición correspondiente del vector es +1 y cuando es blanco es -1. Cada patrón es asociado a una variable de salida que toma el valor de +1 cuando el patrón debe ser reconocido y 0 cuando debe ser ignorado. De esta forma, el problema puede plantearse como:

         Entrada    Salida
      (x0, x1, x2)                      +----+
     -----------------------            | x0 |
           000        0                 +----+
           001        1                 | x1 | 
           010        0                 +----+
           011        0                 | x2 |
           100        1                 +----+
           101        0
           110        1
           111        1       

In [1]:
%clear
##
## A continuación se codifica la red que soluciona el problema
## anterior (no es generica).
##

import tensorflow as tf
import numpy as np

def nnet(x):
    """
    Computa el valor de la red neuronal para el vector
    de entrada x
    
    >>> import tensorflow as tf
    >>> import numpy as np
    >>> x = [[0, 0, 0],
    ...      [0, 0, 1],
    ...      [0, 1, 0],
    ...      [0, 1, 1],
    ...      [1, 0, 0],
    ...      [1, 0, 1],
    ...      [1, 1, 0],
    ...      [1, 1, 1]]
                      
    >>> for z in x:
    ...     nnet(z)
    0.0
    1.0
    0.0
    0.0
    1.0
    0.0
    1.0
    1.0
    """
    
    ## entrada a la red neuronal
    z = tf.placeholder(tf.float32, shape=(3, 1))
    
    ## peso de las neuronas inhibitorias
    N = -10.0
    
    ## pesos asociados a las conexiones 
    weights = tf.constant([[N, N, 1],    ## 001
                           [1, N, N],    ## 100
                           [1, 1, N],    ## 110
                           [1, 1, 1]])   ## 111
    
    ## valor límite de las neuronas para 
    ## la función de activación
    threshold = tf.constant([[1.], [1.], [2.], [3.]])
    
    ## operaciones matemáticas
    out = tf.matmul(weights, z)
    out = tf.subtract(out, threshold)
    out = tf.map_fn(lambda m: tf.cond(tf.greater_equal(m[0], 0.0), 
                                      true_fn = lambda: tf.constant(1.), 
                                      false_fn = lambda : tf.constant(0.0)),
                    out)
    out = tf.reduce_max(out)
    
    with tf.Session() as sess:
        x = np.array(x).reshape(3,1)
        out = sess.run(out, feed_dict={z: x})
    return out


if __name__ == "__main__":
    import doctest
    doctest.testmod()

/Users/jdvelasq/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


---
**Ejercicio.--** Complete el siguiente código que implementa una red de neuronas de McCulloch-Pitts. Realice la implementación usando `numpy`. 

In [8]:
%clear
import numpy as np

class McCullochPits:
    z = np.array([])
    holi = np.array([])
    n = 0
    def __init__(self, n):
        #self.z =
        self.n = -n*2
        """
        n es el numero de entradas a la red
        """
        pass

    def fit(self, x, y):
        self.holi = [[[self.n if i == 0 else i for i in v], np.sum(v)] for v, w in zip(x, y) if w > 0]
        """
        x son las entradas, y es la salida esperada
        """
        pass

    def predict(self, x):
        for e in x:
            cosa = []
            for i in range(len(self.holi)):
                cosa.append(1 if np.sum(np.array(e) * np.array(self.holi[i][0])) >= self.holi[i][1] else 0)
            print(np.max(cosa))

        """
        Pronostica la salida para una lista de entradas.

        >>> x = [[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1],
        ...      [1, 0, 0], [1, 0, 1], [1, 1, 0], [1, 1, 1]]
        >>> y = [0, 1, 0, 0,
        ...      1, 0, 1, 1]
        >>> m = McCullochPits(3)
        >>> m.fit(x, y)
        >>> m.predict(x)
        0
        1
        0
        0
        1
        0
        1
        1
        
        """
        pass

if __name__ == "__main__":
        x = [[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1], [1, 0, 0], [1, 0, 1], [1, 1, 0], [1, 1, 1]]
        y = [0, 1, 0, 0, 1, 0, 1, 1]
        m = McCullochPits(3)
        m.fit(x, y)
        m.predict(x)
        import doctest
        doctest.testmod()

0
1
0
0
1
0
1
1


---

Modelo de McCulloch-Pitts
===

**Juan David Velásquez Henao**  
jdvelasq@unal.edu.co   
Universidad Nacional de Colombia, Sede Medellín  
Facultad de Minas  
Medellín, Colombia

---

Haga click [aquí](https://github.com/jdvelasq/deep-neural-nets/tree/master/) para acceder al repositorio online.

Haga click [aquí](http://nbviewer.jupyter.org/github/jdvelasq/deep-neural-nets/tree/master/) para explorar el repositorio usando `nbviewer`. 